wat.011  https://easy.dans.knaw.nl/ui/datasets/id/easy-dataset:64145/tab/2

format: asc
download 2000 phosphorus and nitrogen total load and retention data see readme.pdf for more info

Import libraries

In [2]:
# Libraries for downloading data from remote server (may be ftp)
import requests
from urllib.request import urlopen
from contextlib import closing
import shutil

# Library for uploading/downloading data to/from S3
import boto3

# Libraries for handling data
import rasterio as rio
import numpy as np
# from netCDF4 import Dataset
# import pandas as pd
# import scipy

# Libraries for various helper functions
# from datetime import datetime
import os
import threading
import sys
from glob import glob

s3 tools

In [3]:
s3_upload = boto3.client("s3")
s3_download = boto3.resource("s3")

s3_bucket = "wri-public-data"
s3_folder = "resourcewatch/raster/wat_011_global_riverine_N_P/"

s3_file1 = "wat_011_global_riverine_N_Load_2000.asc"
s3_file2 = "wat_011_global_riverine_P_Load_2000.asc"
s3_file3 = "wat_011_global_riverine_N_Retention_2000.asc"
s3_file4 = "wat_011_global_riverine_P_Retention_2000.asc"

s3_key_orig1 = s3_folder + s3_file1
s3_key_edit1 = s3_key_orig1[0:-4] + "_edit.tif"

s3_key_orig2 = s3_folder + s3_file2
s3_key_edit2 = s3_key_orig2[0:-4] + "_edit.tif"

s3_key_orig3 = s3_folder + s3_file3
s3_key_edit3 = s3_key_orig3[0:-4] + "_edit.tif"

s3_key_orig4 = s3_folder + s3_file4
s3_key_edit4 = s3_key_orig4[0:-4] + "_edit.tif"

s3_files_to_merge = [s3_key_edit1, s3_key_edit2, s3_key_edit3,s3_key_edit4]
band_ids = ["N_Load","P_Load","N_Retention", "P_Retention"]
s3_key_merge = s3_folder + "Wat_011_global_riverine_Merge_2000.tif"

class ProgressPercentage(object):
        def __init__(self, filename):
            self._filename = filename
            self._size = float(os.path.getsize(filename))
            self._seen_so_far = 0
            self._lock = threading.Lock()

        def __call__(self, bytes_amount):
            # To simplify we'll assume this is hooked up
            # to a single filename.
            with self._lock:
                self._seen_so_far += bytes_amount
                percentage = (self._seen_so_far / self._size) * 100
                sys.stdout.write("\r%s  %s / %s  (%.2f%%)"%(
                        self._filename, self._seen_so_far, self._size,
                        percentage))
                sys.stdout.flush()

Define local file locations

In [4]:
local_folder = "/Users/Max81007/Desktop/Python/Resource_Watch/Raster/wat.011/"

file_name1 = "Nload_2000.asc"
file_name2 = "Pload_2000.asc"
file_name3 = "retention_N_2000.asc"
file_name4 = "retention_P_2000.asc"


local_orig1 = local_folder + file_name1
local_orig2 = local_folder + file_name2
local_orig3 = local_folder + file_name3
local_orig4 = local_folder + file_name4


orig_extension_length = 4 #4 for each char in .tif

local_edit1 = local_orig1[:-orig_extension_length] + "_edit.tif" 
local_edit2 = local_orig2[:-orig_extension_length] + "_edit.tif" 
local_edit3 = local_orig3[:-orig_extension_length] + "_edit.tif" 
local_edit4 = local_orig4[:-orig_extension_length] + "_edit.tif" 

merge_files = [local_edit1, local_edit2, local_edit3, local_edit4]
tmp_merge = local_folder + "Foo_039_Fertilizer_TotalConsumption_Merge.tif"


Use rasterio to reproject and compress

In [4]:
files = [local_orig1, local_orig2, local_orig3, local_orig4]
for file in files:
    with rio.open(file, 'r') as src:
        profile = src.profile
        print(profile)

{'driver': 'AAIGrid', 'dtype': 'float32', 'nodata': -9999.0, 'width': 720, 'height': 360, 'count': 1, 'crs': None, 'transform': Affine(0.5, 0.0, -180.0,
       0.0, -0.5, 90.0), 'tiled': False}
{'driver': 'AAIGrid', 'dtype': 'float32', 'nodata': -9999.0, 'width': 720, 'height': 360, 'count': 1, 'crs': None, 'transform': Affine(0.5, 0.0, -180.0,
       0.0, -0.5, 90.0), 'tiled': False}
{'driver': 'AAIGrid', 'dtype': 'float32', 'nodata': -9999.0, 'width': 720, 'height': 360, 'count': 1, 'crs': None, 'transform': Affine(0.5, 0.0, -180.0,
       0.0, -0.5, 90.0), 'tiled': False}
{'driver': 'AAIGrid', 'dtype': 'float32', 'nodata': -9999.0, 'width': 720, 'height': 360, 'count': 1, 'crs': None, 'transform': Affine(0.5, 0.0, -180.0,
       0.0, -0.5, 90.0), 'tiled': False}


In [5]:
# Note - this is the core of Vizz's netcdf2tif function
def convert_asc_to_tif(orig_name, edit_name):

    with rio.open(orig_name, 'r') as src:
        # This assumes data is readable by rasterio
        # May need to open instead with netcdf4.Dataset, for example

        data = src.read()[0]

        rows = data.shape[0]
        columns = data.shape[1]

        print(rows)
        print(columns)

        # Latitude bounds
        south_lat = -90
        north_lat = 90

        # Longitude bounds
        west_lon = -180
        east_lon = 180

        transform = rio.transform.from_bounds(west_lon, south_lat, east_lon, north_lat, columns, rows)

        # Profile

        no_data_val = -9999.0
        target_projection = 'EPSG:4326'
        target_data_type = np.float64

        profile = {
            'driver':'GTiff', 
            'height':rows, 
            'width':columns, 
            'count':1, 
            'dtype':target_data_type, 
            'crs':target_projection, 
            'transform':transform, 
            'compress':'lzw', 
            'nodata': no_data_val
        }

        with rio.open(edit_name, "w", **profile) as dst:
            dst.write(data.astype(profile["dtype"]), 1)

In [6]:
convert_asc_to_tif(local_orig1, local_edit1)
convert_asc_to_tif(local_orig2, local_edit2)
convert_asc_to_tif(local_orig3, local_edit3)
convert_asc_to_tif(local_orig4, local_edit4)

360
720
360
720
360
720
360
720


In [7]:
with rio.open(merge_files[0]) as src:
    kwargs = src.profile
    
kwargs.update(
    count=len(merge_files)
)
    
with rio.open(tmp_merge, 'w', **kwargs) as dst:
    for idx, file in enumerate(merge_files):
        print(idx)
        with rio.open(file) as src:

            band = idx+1
            windows = src.block_windows()

            for win_id, window in windows:
                src_data = src.read(1, window=window)
                dst.write_band(band, src_data, window=window)

0
1
2
3


In [8]:
files = [tmp_merge]
for file in files:
    with rio.open(file, 'r') as src:
        profile = src.profile
        print(profile)

{'driver': 'GTiff', 'dtype': 'float64', 'nodata': -9999.0, 'width': 720, 'height': 360, 'count': 4, 'crs': CRS({'init': 'epsg:4326'}), 'transform': Affine(0.5, 0.0, -180.0,
       0.0, -0.5, 90.0), 'compress': 'lzw', 'interleave': 'band', 'tiled': False}


Upload orig and edit files to s3

In [9]:
# Original
s3_upload.upload_file(local_orig1, s3_bucket, s3_key_orig1,
                         Callback=ProgressPercentage(local_orig1))
s3_upload.upload_file(local_orig2, s3_bucket, s3_key_orig2,
                         Callback=ProgressPercentage(local_orig2))
s3_upload.upload_file(local_orig3, s3_bucket, s3_key_orig3,
                         Callback=ProgressPercentage(local_orig3))
s3_upload.upload_file(local_orig4, s3_bucket, s3_key_orig4,
                         Callback=ProgressPercentage(local_orig4))


# Edit
s3_upload.upload_file(local_edit1, s3_bucket, s3_key_edit1,
                         Callback=ProgressPercentage(local_edit1))
s3_upload.upload_file(local_edit2, s3_bucket, s3_key_edit2,
                         Callback=ProgressPercentage(local_edit2))
s3_upload.upload_file(local_edit3, s3_bucket, s3_key_edit3,
                         Callback=ProgressPercentage(local_edit3))
s3_upload.upload_file(local_edit4, s3_bucket, s3_key_edit4,
                         Callback=ProgressPercentage(local_edit4))

s3_upload.upload_file(tmp_merge, s3_bucket, s3_key_merge,
           Callback=ProgressPercentage(tmp_merge))


/Users/Max81007/Desktop/Python/Resource_Watch/Raster/wat.011/Foo_039_Fertilizer_TotalConsumption_Merge.tif  1717086 / 1717086.0  (100.00%)

In [5]:
os.environ["Zs3_key"] = "s3://wri-public-data/" + s3_key_merge
os.environ["Zs3_key_inspect"] = "wri-public-data/" + s3_key_merge
os.environ["Zgs_key"] = "gs://resource-watch-public/" + s3_key_merge

!echo %Zs3_key_inspect%
!aws s3 ls %Zs3_key_inspect%

wri-public-data/resourcewatch/raster/wat_011_global_riverine_N_P/Wat_011_global_riverine_Merge_2000.tif
2017-09-28 10:13:32    1717086 Wat_011_global_riverine_Merge_2000.tif


In [6]:
!gsutil cp %Zs3_key% %Zgs_key%

Copying s3://wri-public-data/resourcewatch/raster/wat_011_global_riverine_N_P/Wat_011_global_riverine_Merge_2000.tif [Content-Type=binary/octet-stream]...
/ [0 files][    0.0 B/  1.6 MiB]                                                
/ [1 files][  1.6 MiB/  1.6 MiB]                                                
-

Operation completed over 1 objects/1.6 MiB.                                      


In [7]:
os.environ["asset_id"] = "users/resourcewatch/wat_011_global_riverine_N_P_2000"
!earthengine upload image --asset_id=%asset_id% %Zgs_key%

Started upload task with ID: SWXUHWM45AWV7QV4FB4M2POY


In [8]:
os.environ["band_names"] = str(band_ids)
!earthengine asset set -p band_names="%band_names%" %asset_id%